In [1]:
import numpy as np
import pandas as pd


from sklearn import datasets
from sklearn.datasets import make_blobs

from scipy import spatial
from NPIR import NPIR

import datetime
import warnings
from sklearn import metrics
from collections import Counter as Cs
from timeit import default_timer as timer
from datetime import timedelta
import matplotlib.pyplot as plt


import findspark
findspark.init()
from pyspark import SparkContext
from operator import *
from pyspark import StorageLevel

from pyspark.sql.types import IntegerType, FloatType, BooleanType, StringType, StructType,\
StructField,ArrayType
from pyspark.sql.functions import udf, log, rand, monotonically_increasing_id, col, broadcast,\
greatest,desc,asc, row_number, avg, mean, least, struct, lit, sequence
import pyspark.sql.functions as F

from pyspark.sql import SparkSession, SQLContext, Window, Row
from pyspark import SparkConf


In [2]:
# spark = SparkSession.builder.master("local[*]").config("spark.sql.broadcastTimeout", "30000s").\
# config("spark.network.timeout","30000s").config("spark.executor.heartbeatInterval","12000000ms").\
# config("spark.storage.blockManagerSlaveTimeoutMs","12000001ms").config("spark.driver.maxResultSize","5g").\
# config("spark.default.parallelism", "100").config("spark.memory.offHeap.enabled","true").\
# config("spark.memory.offHeap.size", "16g").appName("mykmeans").getOrCreate()


In [2]:
# sc = spark.sparkContext
# sqlContext = SQLContext(sc)

In [4]:
# #read csv
# data_spark_df = spark.read.format('csv').option('header','False').option('index','False').load('seeds.csv')

# data_spark_df

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string]

In [5]:
# data_spark_df.describe()

DataFrame[summary: string, _c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string]

In [6]:
# data_spark_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [7]:
# data_spark_df = data_spark_df.select('_c3', '_c4')

In [8]:
# new_name = ['first', 'second']
# data_spark_df = data_spark_df.toDF(*new_name)

# data_spark_df.show()

+-----+------+
|first|second|
+-----+------+
|5.763| 3.312|
|5.554| 3.333|
|5.291| 3.337|
|5.324| 3.379|
|5.658| 3.562|
|5.386| 3.312|
|5.563| 3.259|
| 5.42| 3.302|
|6.053| 3.465|
|5.884| 3.505|
|5.714| 3.242|
|5.438| 3.201|
|5.439| 3.199|
|5.479| 3.156|
|5.482| 3.114|
|5.351| 3.333|
|5.119| 3.383|
|5.527| 3.514|
|5.205| 3.466|
|5.226| 3.049|
+-----+------+
only showing top 20 rows



In [26]:
# spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

# spark.conf.set('spark.jars.packages','com.databricks:spark-cav_2.11')

# spark.conf.set("spark.sql.parquet.compression.codec","gzip")

# spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# sqlContext.setConf("spark.sql.shuffle.partitions", "10")

# sqlContext.sql("SET spark.sql.autoBroadcastJoinThreshold = -1")

DataFrame[key: string, value: string]

In [27]:
# data_spark_df = data_spark_df.withColumn("first_numeric", data_spark_df["first"].cast(FloatType()))
# data_spark_df = data_spark_df.withColumn("second_numeric", data_spark_df["second"].cast(FloatType()))
# data_spark_df = data_spark_df.drop('first').drop('second')
# data_spark_df

DataFrame[first: string, second: string, first_numeric: float]

In [ ]:
data 

In [3]:
#IR: The indexing ratio to be used for generating the maximum index
IR = 0.2
#The number of iteration i
i = 10
k = 3 #k: Number of clusters
# count = Cs()

In [5]:
data = pd.read_csv('blobs.csv')

Execution time HH:MM:SS: 0:00:00.015702


In [ ]:
start = timer()
label = NPIR(data.values[:,:-1],k,IR,i)
end = timer()
print ("Execution time HH:MM:SS:",timedelta(seconds=end-start))

In [81]:
# maxIndex = IR * data_spark_df.count()**2

In [ ]:
# data_spark_df.foreachPartition

In [82]:
# type(data_spark_df)

pyspark.sql.dataframe.DataFrame

In [83]:
# (34579087654324567890//1000)+1

34579087654324568